# Generate Test Structure Names

## Setup

### Imports

In [1]:
from typing import Dict, List, Union

from pathlib import Path
import re
import xml.etree.ElementTree as ET
from itertools import chain

import pandas as pd
import xlwings as xw


### Paths

In [2]:
reference_path = Path.cwd()


## TG263 examples for testing

### Target examples from task group text


In [3]:
target_examples = ['PTVn', 'GTVp', 'CTVsb', 'GTVpar', 'CTVv', 'CTVvas',
                   'PTV1', 'PTV2', 'GTVp1', 'GTVp2', 'PTVp1_CT1PT1', 'GTV_CT2',
                   'CTV_A_Aorta', 'CTV_A_Celiac', 'GTV_Preop', 'PTV_Boost',
                   'PTV_Eval', 'PTV_MR2_Prostate', 'PTV_High', 'CTV_Mid',
                   'PTV_Low', 'PTV_Low', 'PTV_Mid01', 'PTV_Mid02', 'PTV_Mid03',
                   'PTV_High', 'PTV_5040', 'PTV_50.4Gy', 'PTV_50p4Gy',
                   'PTV_Liver_2000x3', 'PTV_Liver_20Gyx3', 'PTV^Physician1',
                   'GTV_Liver^ICG', 'PTV_Eval_7000-08', 'PTV-03', 'CTVp2-05',
                   'zPTVopt']


### Non-Target examples from task group text

In [4]:
non_target_examples = ['Lungs', 'Kidneys', 'Hippocampi', 'LNs', 'Ribs_L',
                       'Femur_Head', 'Ears_External', 'Bowel_Bag', 'Lung_L',
                       'Lung_LUL', 'Lung_RLL', 'OpticNrv_PRV03_L', 'SeminalVes',
                       'SeminalVes_Dist', 'A_Aorta', 'A_Carotid', 'V_Portal',
                       'V_Pulmonary', 'LN_Ax_L1', 'LN_IMN', 'CN_IX_L',
                       'CN_XII_R', 'Glnd_Submand', 'Bone_Hyoid', 'Bone_Pelvic',
                       'Musc_Masseter', 'Musc_Sclmast_L', 'Spc_Bowel',
                       'Spc_Retrophar_L', 'Sinus_Frontal', 'Sinus_Maxillary',
                       'Brainstem_PRV', 'SpinalCord_PRV05', 'Brainstem_PRV03',
                       'OpticChiasm_PRV3', 'Brain~', 'Lung~_L', 'Lungs^Ex',
                       'CaudaEquina', 'OpticChiasm', 'A_Mesenteric',
                       'A_Illiac', 'CN_I', 'Parotid']


### Structure names from TG263_Nomenclature_Worksheet_20170815

In [5]:
file_name = 'TG263_Nomenclature_Worksheet_20170815.csv'
file_path = reference_path / file_name
name_table = pd.read_csv(file_path)
names = name_table['TG263-Primary Name'].drop_duplicates()
names.dropna(inplace=True)
table_examples = list(names)


### Structure names from the TG263 Eclipse Templates

**Xpath searches for Structure Templates**

> ```
> /StructureTemplate/Structures/Structure
> /StructureTemplate/Structures/Structure/@ID
> /StructureTemplate/Structures/Structure/Identification/VolumeType
> /StructureTemplate/Structures/Structure/Identification/StructureCode/@Code
> /StructureTemplate/Structures/Structure/Identification/StructureCode/@CodeScheme
> ```

In [6]:
names = []
template_folder = reference_path / 'EclipseStructureTemplates'
for template_file in template_folder.glob('*.xml'):
    tree = ET.parse(template_file)
    root = tree.getroot()
    for struct in root.findall(r'.//Structure'):
        volume = struct.find(r'./Identification/VolumeType')
        if volume is not None:
            volume_type = volume.text
        else:
            volume_type = None
        structure_code = struct.find(r'./Identification/StructureCode')
        if structure_code is not None:
            s_code = structure_code.attrib['Code']
            s_scheme = structure_code.attrib['CodeScheme']
        else:
            s_code = None
            s_scheme = None
        structure = {
            'Name': struct.attrib['ID'],
            'VolumeType': volume_type,
            'Code': s_code,
            'Scheme': s_scheme
            }
        names.append(structure)
template_structures = pd.DataFrame(names)
#xw.view(template_structures)
names = list(template_structures.Name)


### Merge all of the structure names

In [7]:
def replace_num_placeholders(text:str)->str:
    '''Replace number placeholders with numbers for testing.

    Replace 'xxxx' with '5040'
    Replace 'xx' with '09'
    Replace 'PRVx' with 'PRV5' (Just 'x' will catch fractions designators.)
    Remove leading and trailing spaces.

    Args:
        text (str): Text to clean

    Returns:
        str: Cleaned text
    '''
    txt4 = text.replace('xxxx', '5040')
    txt2 = txt4.replace('xx', '09')
    txt1 = txt2.replace('PRVx', 'PRV5')
    txt_final = txt1.strip()
    return txt_final


examples = [replace_num_placeholders(name)
            for name in chain(target_examples,
                               non_target_examples,
                               table_examples,
                               names)]


### Save the list to a text file

In [8]:
Path('examples.txt').write_text('\n'.join(examples))


11038